# Notebook 3: Vector operations

In this notebook we will look at four vector operations, and how these are very useful to solve different types of problems in structural geology:

## Scalar multiplication:

To multiply a scalar times a vector, just multiply each component of the vector by the scalar:

$$x\mathbf v=[xv_1,xv_2,xv_3]$$

This operation is useful to reverse the direction of a vector, just multiply the vector by -1.

## Addition

To add two vectors, just sum their components. This operation is commutative:

$$\mathbf u + \mathbf v=\mathbf v+\mathbf u=[u_1+v_1,u_2+v_2,u_3+v_3]$$

## Dot product

The result of the dot product is a scalar and is equal to the magnitude of the first vector times the magnitude of the second vector times the cosine of the angle between the vectors. The dot product is commutative:

$$\mathbf u \cdot\mathbf v=\mathbf v \cdot\mathbf u=uv\cos\theta=u_1v_1+u_2v_2+u_3v_3=u_iv_i$$

If the two vectors are unit vectors, you can see that the dot product is the cosine of the angle between the vectors:

$$\mathbf{\hat u} \cdot\mathbf{\hat v}=\cos\theta=u_1v_1+u_2v_2+u_3v_3$$

or in terms of the direction cosines of the vectors:

$$\mathbf{\hat u}\cdot\mathbf{\hat v}=\cos\theta=cos\alpha_1\cos\alpha_2+\cos\beta_1\cos\beta_2+\cos\gamma_1\cos\gamma_2$$

which is a great way to find the angle between two unit vectors (two lines).

## Cross product

The result of the cross product is another vector. This vector is perpendicular to the other two vectors, and it has a magnitude equal to the product of the magnitudes of the vectors times the sine of the angle between the vectors:

$$\mathbf u\times\mathbf v=uv\sin\theta\mathbf{\hat l}=[u_2v_3-u_3v_2,u_3v_1-u_1v_3,u_1v_2-u_2v_1]$$

The cross product is not commutative. If the vectors are unit vectors, the length of the resulting vector is equal to the sine of the angle between the vectors. The new vector obeys a right-hand rule (see figure below); for $\mathbf u\times\mathbf v$, the fingers curl from $\mathbf u$ towards $\mathbf v$ and the thumb points in the direction of the resulting vector, and vice versa:

<img src="../figures/cross_product.png" alt="cross_product" width="500" style="display: block; margin: 0 auto"/><br><br>

The cross product is great for finding the vector perpendicular to two non-parallel vectors. Since two non-parallel vectors (or lines) define a plane, this translates to finding the pole to a plane containing two non-parallel lines.

## Orientation problems

The vector operations above are useful to solve a great deal of orientation problems in structural geology. These are summarized in the table below:

<img src="../figures/orientation_problems.png" alt="orientation_problems" width="700" style="display: block; margin: 0 auto"/><br><br>

In addition, vector addition is useful to find the mean of a group of vectors. We will look at that later.

## Python functions

The following Python functions solve the problems in the table above:

In [1]:
# A group of Python functions to solve orientation problems. These are also in functions/angles.py

import numpy as np

# this makes visible our functions folder
import sys, os
sys.path.append(os.path.abspath("../functions"))

from sph_to_cart import sph_to_cart
from cart_to_sph import cart_to_sph
from pole import pole_from_plane, plane_from_pole

# Python functions based on the Matlab function
# Angles in Allmendinger et al. (2012)

def angle_bw_lines(trd1, plg1, trd2, plg2):
    """
    angle_bw_lines returns the angle between two lines
    of trend and plunge trd1, plg1, trd2, and plg2
    Input and output angles are in radians
    """
    # convert lines to directions cosines and numpy arrays
    cn1, ce1, cd1 = sph_to_cart(trd1, plg1)
    u = np.array([cn1, ce1, cd1])
    cn2, ce2, cd2 = sph_to_cart(trd2, plg2)
    v = np.array([cn2, ce2, cd2])
    # angle between lines is arccosine of their dot product
    return np.arccos(np.dot(u, v))

def angle_bw_planes(str1, dip1, str2, dip2):
    """
    angle_bw_planes returns the angle between two planes
    of strike and dip str1, dip1, str2, and dip2
    Input and output angles are in radians
    """
    # compute poles to lines
    pole1_trd, pole1_plg = pole_from_plane(str1, dip1)
    pole2_trd, pole2_plg = pole_from_plane(str2, dip2)
    # find angle between poles
    angle = angle_bw_lines(pole1_trd, pole1_plg, pole2_trd, pole2_plg)
    # angle between planes is the complementary angle
    return (np.pi - angle)

def pole_from_lines(trd1, plg1, trd2, plg2):
    """
    pole_from_lines compute the pole to a plane given
    two lines on the plane, with trend and plunge trd1, plg1,
    trd2, and plg2
    Input and output angles are in radians
    """
    # convert lines to direction cosines and numpy arrays
    cn1, ce1, cd1 = sph_to_cart(trd1, plg1)
    u = np.array([cn1, ce1, cd1])
    cn2, ce2, cd2 = sph_to_cart(trd2, plg2)
    v = np.array([cn2, ce2, cd2])
    # normal is cross product between vectors
    pole = np.cross(u, v)
    # make pole a unit vector
    norm = np.linalg.norm(pole)
    pole = pole/norm
    # if pole points upwards, make it point downwards
    if pole[2] < 0:
        pole *= -1.0
    # return trend and plunge of pole
    return cart_to_sph(pole[0], pole[1], pole[2])

def plane_from_app_dips(trd1, plg1, trd2, plg2):
    """
    plane_from_app_dips returns the strike and dip of a plane
    from two apparent dips with trend and plunge trd1, plg1,
    trd2, and plg2
    Input and output angles are in radians
    """
    # Compute pole to plane from apparent dips (lines)
    pole_trd, pole_plg = pole_from_lines(trd1,plg1,trd2,plg2)
    # return strike and dip of plane
    return plane_from_pole(pole_trd, pole_plg)

def int_bw_planes(str1, dip1, str2, dip2):
    """
    int_bw_planes returns the intersection between two planes
    of strike and dip str1, dip1, str2, dip2
    Input and output angles are in radians
    """
    # compute poles to planes
    pole1_trd, pole1_plg = pole_from_plane(str1, dip1)
    pole2_trd, pole2_plg = pole_from_plane(str2, dip2)
    # intersection is normal to poles
    return pole_from_lines(pole1_trd, pole1_plg, pole2_trd, pole2_plg)

## Applications

Let’s start with the following problem from Lisle and Leyshon (2004): Two limbs of a chevron fold (A and B) have orientations (RHR) as follows:

Limb A = 120/40 \
Limb B = 250/60 

Determine: 

1. the trend and plunge of the hinge line of the fold, 
2. the rake of the hinge line in limb A, 
3. the rake of the hinge line in limb B

In [2]:
# Solution:

# Strike and dip of the limbs in radians
str1, dip1 = np.radians([120, 40])
str2, dip2 = np.radians([250, 60])

# 1. Chevron folds have planar limbs. The hinge of the fold
#    is the intersection of the limbs
htrd, hplg = int_bw_planes(str1, dip1, str2, dip2)
print(f"Hinge trend = {np.rad2deg(htrd):.1f}, plunge = {np.rad2deg(hplg):.1f}")

# The rake of the hinge on either limb is the angle between the strike line
# on the limb and the hinge. The strike line is horizontal
plg = 0

# 2. Rake of hinge in limb A
ang = angle_bw_lines(str1, plg, htrd, hplg)
print(f"Rake of hinge in limb A = {np.rad2deg(ang):.1f} E")

# 3. Rake of hinge in limb B
ang = angle_bw_lines(str2, plg, htrd, hplg)
print(f"Rake of hinge in limb A = {np.rad2deg(ang):.1f} W")

Hinge trend = 265.8, plunge = 25.3
Rake of hinge in limb A = 138.4 E
Rake of hinge in limb A = 29.5 W


Let’s do another problem from the same book: A quarry has two walls, one trending 002° and the other 135°. The apparent dip of bedding on the faces are 40° N and 30° SE respectively. Calculate the strike and dip of bedding.

In [3]:
# Solution:

# The apparent dips are just two lines on bedding
trd1, plg1 = np.radians([2, 40])
trd2, plg2 = np.radians([135, 30])

# Calculate bedding from the apparent dips
strike, dip = plane_from_app_dips(trd1,plg1,trd2,plg2)
print(f"Bedding strike = {np.rad2deg(strike):.1f}, dip = {np.rad2deg(dip):.1f}")

Bedding strike = 333.9, dip = 60.7


And the final problem from the same book: A fold has a hinge line oriented 300/40 (trend/plunge). On a map of this structure, the trace of the fold's axial plane trends 088. Calculate the strike and dip of the axial plane:

In [4]:
# Solution:

# This problem is similar to the one before. The axial plane contains 
# the hinge line and the horizontal axial plane trace on the map
trd1, plg1 = np.radians([300, 40])
trd2, plg2 = np.radians([88, 0])

# Calculate axial plane from the two lines
strike, dip = plane_from_app_dips(trd1,plg1,trd2,plg2)
print(f"Axial plane strike = {np.rad2deg(strike):.1f}, dip = {np.rad2deg(dip):.1f}")

Axial plane strike = 268.0, dip = 57.7


Isn't this fun? We now have a set of Python functions to solve many orientation problems in structural geology. 